# Load Dependencies

In [ ]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
import re
from scipy.stats import rankdata, spearmanr, f_oneway, kruskal
import scipy.stats as stats

# Load Data and Process

In [ ]:
def extract_syndromic_dx(response):
    # Define the regex pattern to match the confidence level
    pattern = r'\*\*syndromic diagnosis:\*\* (Normal|MCI|Dementia)'
    # Search for the pattern in the response
    match = re.search(pattern, response, re.IGNORECASE)
    # Extract and return the confidence level if found
    if match:
        return match.group(1).lower()  # Convert to lowercase for consistency
    else:
        return None
    
def extract_confidence_level(response):
    # Use regular expression to find the confidence level
    match = re.search(r'\*\*confidence level:\*\*\s*(\d+)', response)
    
    if match:
        confidence_level = int(match.group(1))
        return confidence_level
    else:
        return None  
    
# Function to load all JSON files and combine them into a single DataFrame
def load_patient_data(json_directory):
    responses = []
    predicted_dx = []
    patient_ids = []
    conf_level = []
    # Iterate over all JSON files in the specified directory
    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            filepath = os.path.join(json_directory, filename)
            with open(filepath, 'r') as file:
                data_dict = json.load(file)
                for patient_id, result in data_dict.items():
                    responses.append(result)
                    predicted_dx.append(extract_syndromic_dx(result))
                    patient_ids.append(patient_id)
                    conf_level.append(extract_confidence_level(result))
    # Combine all the loaded data into a single DataFrame
    combined_data = pd.DataFrame(
        {"PatientID": patient_ids,
        "ResponseTXT": responses,
        "predicted_dx": predicted_dx,
        "confidence_level": conf_level}
    )

    return combined_data

# Load the patient data from the JSON files in the specified directory
path = "../Results - sydronmic dx/GPT4o - Attempt 4"

df_result_attempt4 = load_patient_data(path)
df_result_attempt4.head()

In [ ]:
# Identify the rows where the predicted diagnosis is not one of the expected values
df_result_attempt4[~df_result_attempt4['predicted_dx'].isin(['normal', 'mci', 'dementia'])]

In [ ]:
# Manually correct the predicted diagnoses for specific PatientIDs
df_result_attempt4.loc[df_result_attempt4['PatientID']=='XXX', 'predicted_dx'] = 'normal'
df_result_attempt4.loc[df_result_attempt4['PatientID']=='YYY', 'predicted_dx'] = 'dementia'
df_result_attempt4.loc[df_result_attempt4['PatientID']=='ZZZ', 'predicted_dx'] = 'mci'

In [ ]:
df_dx = pd.read_csv('../../EDW Queries/Moura et al data-all-notes/patient_dx.csv')
df_result = df_result_attempt4.merge(df_dx, on='PatientID', how='inner')

In [ ]:
def convert_dx_to_category(value):
    if value == 0:
        return 'CU'
    elif value == 2:
        return 'MCI'
    elif value == 4:
        return 'Dementia'
    
def standardize_dx(value):
    if value == 'normal':
        return 'CU'
    elif value == 'mci':
        return 'MCI'
    elif value == 'dementia':
        return 'Dementia'

df_result['actual_category'] = df_result['syndromic_dx'].apply(convert_dx_to_category)

df_result['predicted_category'] = df_result['predicted_dx'].apply(standardize_dx)

# Analyses and Visualizations

## Performance Evaluation

### Confusion Matrix

In [ ]:
all_labels = ['CU','MCI','Dementia']

# Compute the confusion matrix
cm = confusion_matrix(df_result['actual_category'], df_result['predicted_category'], labels=all_labels)

# Normalize the confusion matrix by dividing each row by the sum of the row
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Create annotation strings for counts and percentages
annot = np.array([["{0}\n({1:.1%})".format(count, perc) for count, perc in zip(row_counts, row_percents)] 
                  for row_counts, row_percents in zip(cm, cm_normalized)])

# Convert the confusion matrix into a DataFrame for easier plotting
cm_df = pd.DataFrame(cm, index=all_labels, columns=all_labels)
# Set font style
plt.rcParams['font.family'] = 'Arial'

# Plot the heatmap with both counts and percentages
plt.figure(figsize=(10, 8))
ax = sns.heatmap(cm, annot=annot, cmap="YlGnBu", fmt="", annot_kws={"size": 32}, xticklabels=all_labels, yticklabels=all_labels)

plt.xlabel('Predicted', fontsize=35)  
plt.ylabel('Actual', fontsize=35)  
plt.xticks(fontsize=32)  
plt.yticks(fontsize=32)  
colorbar = ax.collections[0].colorbar  # Access the color bar
colorbar.ax.tick_params(labelsize=24)  # Set font size for color bar tick labels

plt.show()

### Evaluation Metrics

In [ ]:
label_mapping = {'CU': 0, 'MCI': 1, 'Dementia': 2}

# Use .loc to avoid SettingWithCopyWarning
df_result.loc[:, 'actual_mapped'] =df_result['actual_category'].map(label_mapping)
df_result.loc[:, 'predicted_mapped'] =df_result['predicted_category'].map(label_mapping)

# Calculate Mean Absolute Error
mae = (df_result['actual_mapped'] -df_result['predicted_mapped']).abs().mean()

# Calculate Quadratic Weighted Kappa
qwk = cohen_kappa_score(df_result['actual_mapped'],df_result['predicted_mapped'], weights='quadratic')

print(f"Mean Absolute Error: {mae}")
print(f"Quadratic Weighted Kappa: {qwk}")

In [ ]:
# Extract true and predicted labels
y_true = df_result['actual_category']
y_pred = df_result['predicted_category']

# Generate classification report
report = classification_report(
    y_true, 
    y_pred, 
    labels=['CU', 'MCI', 'Dementia'],  # ensure consistent order
    output_dict=True
)

report_df = pd.DataFrame(report).transpose()

# Display the per-class metrics
print(report_df[['precision', 'recall', 'f1-score', 'support']])

## Confidence Level

### Adjudicator Confidence

In [ ]:
df_result[df_result['confidence_level'].isna()]
df_result.loc[df_result['PatientID']=='XXX', 'confidence_level'] = 90.0
df_result.loc[df_result['PatientID']=='YYY', 'confidence_level'] = 85.0
df_result.loc[df_result['PatientID']=='ZZZ', 'confidence_level'] = 70.0

In [ ]:
df_certainty = pd.read_csv('../../../../R03/Prelim_Data_R/Dementia_ReferenceStandardDataset_08292019_subset_mrn.csv')
df_id = pd.read_csv('../../../EDW Utilities/folder for JH-patient-features/jh_1000_id.csv')
df_certainty = df_certainty[['empi','syndromic_dx_certainty']].drop_duplicates().merge(df_id, left_on='empi',right_on='EMPI', how = 'left').dropna()[['PatientID','syndromic_dx_certainty']]
df_result = df_result.merge(df_certainty, on='PatientID', how='left')

In [ ]:
counts = df_result['syndromic_dx_certainty'].value_counts().sort_index()

# Calculate the actual confidence distribution
actual_confidence_distribution = counts / counts.sum()

# Calculate the cumulative distribution
cumulative_distribution = np.cumsum(actual_confidence_distribution)

print("Actual Confidence Distribution:\n", actual_confidence_distribution)
print("Cumulative Distribution:\n", cumulative_distribution)

In [ ]:
sns.boxplot(data=df_result, x='actual_category', y='syndromic_dx_certainty')
plt.title("Adjudicator Confidence Score by Diagnosis Category")
plt.show()

cu_scores = df_result[df_result['actual_category'] == 'CU']['syndromic_dx_certainty']
mci_scores = df_result[df_result['actual_category'] == 'MCI']['syndromic_dx_certainty']
dementia_scores = df_result[df_result['actual_category'] == 'Dementia']['syndromic_dx_certainty']

h_stat, p_kw = kruskal(cu_scores, mci_scores, dementia_scores)
print(f"Kruskal-Wallis: H = {h_stat:.3f}, p = {p_kw:.3f}")

### Quantile Mapping

In [ ]:
df_result['confidence_level'] = df_result['confidence_level'].astype(float)
gpt_confidence_scores = np.array(df_result['confidence_level'].tolist())

# Rank the GPT scores to compute quantiles
gpt_ranks = rankdata(gpt_confidence_scores, method='average') / len(gpt_confidence_scores)

# Map GPT quantiles to actual confidence levels
def map_quantile_to_level(gpt_rank):
    if gpt_rank <= cumulative_distribution.iloc[0]:
        return 1
    elif gpt_rank <= cumulative_distribution.iloc[1]:
        return 2
    elif gpt_rank <= cumulative_distribution.iloc[2]:
        return 3
    else:
        return 4

# Apply the mapping
mapped_confidence_levels = np.array([map_quantile_to_level(rank) for rank in gpt_ranks])

for gpt_score, mapped_level in zip(gpt_confidence_scores, mapped_confidence_levels):
    print(f"GPT Score: {gpt_score}, Mapped Confidence Level: {mapped_level}")
df_result.loc[:, 'pred_conf_level_mapped'] = mapped_confidence_levels
df_result['pred_conf_level_mapped'].value_counts()

In [ ]:
df_result['syndromic_dx_certainty_int'] = df_result['syndromic_dx_certainty'].astype(int)
df_result['syndromic_dx_certainty_str'] = df_result['syndromic_dx_certainty_int'].astype(str)
df_result['syndromic_dx_certainty_str'] = pd.Categorical(df_result['syndromic_dx_certainty_str'], 
                                                         categories=['1', '2', '3', '4'], 
                                                         ordered=True)
df_result['pred_dx_certainty_int'] = df_result['pred_conf_level_mapped'].astype(int)
df_result['pred_dx_certainty_str'] = df_result['pred_dx_certainty_int'].astype(str)
df_result['pred_dx_certainty_str'] = pd.Categorical(df_result['pred_dx_certainty_str'], 
                                                         categories=['1', '2', '3', '4'], 
                                                         ordered=True)

In [ ]:
confusion_matrix_data = pd.crosstab(df_result['syndromic_dx_certainty_str'], df_result['pred_dx_certainty_str'])

plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=(10, 8))

ax = sns.heatmap(confusion_matrix_data, annot=True, cmap="YlGnBu", fmt="d", annot_kws={"size": 34})
ax.invert_yaxis()
plt.xlabel('GPT Label', fontsize=35)  
plt.ylabel('Physician Label', fontsize=35)  
plt.xticks(fontsize=32) 
plt.yticks(fontsize=32) 
colorbar = ax.collections[0].colorbar  
colorbar.ax.tick_params(labelsize=24) 

plt.show()

### Stratification Analysis

In [ ]:
def calculate_weighted_kappa_by_certainty(df):
    kappa_scores = {}
    # Loop through each certainty level
    for certainty_level in sorted(df['syndromic_dx_certainty_int'].unique()):
        df_subset = df[df['syndromic_dx_certainty_int'] == certainty_level]
        # Calculate the weighted kappa score for this subset
        kappa_score = cohen_kappa_score(
            df_subset['actual_mapped'], 
            df_subset['predicted_mapped'], 
            weights='quadratic'
        )
        kappa_scores[certainty_level] = kappa_score

    return kappa_scores

kappa_scores_by_certainty = calculate_weighted_kappa_by_certainty(df_result)

for certainty_level, kappa_score in kappa_scores_by_certainty.items():
    print(f"Certainty Level {certainty_level}: Weighted Kappa = {kappa_score:.4f}")


In [ ]:
kappa_df_gpt = pd.DataFrame(list(kappa_scores_by_certainty.items()), columns=['Certainty Level', 'Weighted Kappa'])

plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Certainty Level', y='Weighted Kappa', data=kappa_df_gpt, hue='Certainty Level', palette='YlGnBu', dodge=False)

plt.xlabel('GPT Confidence Score', fontsize=32)
plt.ylabel('Weighted Kappa Score', fontsize=32)
plt.xticks(fontsize=30)  
plt.yticks(fontsize=30)  
ax.set_ylim(0, 1.10) 

# Add text labels on each bar
for i, row in kappa_df_gpt.iterrows():
    ax.text(i, row['Weighted Kappa'] + 0.005, f'{row["Weighted Kappa"]:.2f}', ha='center', va='bottom', fontsize=30)

plt.show()



# Revision - Logprobs

## Load data and process for logprobs

In [ ]:
# Load JSON data from file
with open("../Results - sydronmic dx/GPT4o - Attempt 7 - Revision/patient_group_1_logprobs.json", "r") as f:
    data = json.load(f)

# Prepare lists
ids = []
predictions = []
probabilities = []

# Extract the data
for pid, result in data.items():
    ids.append(pid)
    print(result)
    predictions.append(result["prediction"])
    
    # Find probability_percent for the predicted token
    pred_token = result["prediction"]
    matched_prob = next((item["probability_percent"] for item in result["logprobs"] if item["token"] == pred_token), None)
    probabilities.append(matched_prob)

# Print result
print("IDs:", ids)
print("Predictions:", predictions)
print("Probabilities:", probabilities)


In [ ]:
def load_logprob_json(json_directory):
    ids = []
    predictions = []
    probabilities = []
    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            filepath = os.path.join(json_directory, filename)
            with open(filepath, "r") as f:
                data = json.load(f)
            # Extract the data
            for pid, result in data.items():
                ids.append(pid)
                predictions.append(result["prediction"])
                
                # Find probability_percent for the predicted token
                pred_token = result["prediction"]
                matched_prob = next((item["probability_percent"] for item in result["logprobs"] if item["token"] == pred_token), None)
                probabilities.append(matched_prob)
    combined_data = pd.DataFrame(
        {"PatientID": ids,
        "predicted_dx": predictions,
        "probability": probabilities}
    )

    return combined_data


In [ ]:
df_logprobs = load_logprob_json("../Results - sydronmic dx/GPT4o - Attempt 7 - Revision")

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(df_logprobs['probability'].tolist(), bins=20, edgecolor='black')
plt.title('Distribution of Prediction Probabilities')
plt.xlabel('Probability (%)')
plt.ylabel('Number of Predictions')
plt.grid(True)
plt.show()


In [ ]:
df_dx = pd.read_csv('../../EDW Queries/Moura et al data-all-notes/patient_dx.csv')
df_combined = df_logprobs.merge(df_dx, on='PatientID', how='inner')

In [ ]:
def pred_dx_to_category(value):
    if value == '1':
        return 'CU'
    elif value == '2':
        return 'MCI'
    elif value == '3':
        return 'Dementia'
    
def actual_dx_to_category(value):
    if value == 0:
        return 'CU'
    elif value == 2:
        return 'MCI'
    elif value == 4:
        return 'Dementia'

df_combined['actual_category'] = df_combined['syndromic_dx'].apply(actual_dx_to_category)

df_combined['predicted_category'] = df_combined['predicted_dx'].apply(pred_dx_to_category)

In [ ]:
counts = df_combined['syndromic_dx_certainty'].value_counts().sort_index()

# Calculate the actual confidence distribution
actual_confidence_distribution = counts / counts.sum()

# Calculate the cumulative distribution
cumulative_distribution = np.cumsum(actual_confidence_distribution)

print("Actual Confidence Distribution:\n", actual_confidence_distribution)
print("Cumulative Distribution:\n", cumulative_distribution)

## Analyses

### Quantile Mapping

In [ ]:
gpt_confidence_scores = np.array(df_combined['probability'].tolist())

gpt_ranks = rankdata(gpt_confidence_scores, method='average') / len(gpt_confidence_scores)

def map_quantile_to_level(gpt_rank):
    if gpt_rank <= cumulative_distribution.iloc[0]:
        return 1
    elif gpt_rank <= cumulative_distribution.iloc[1]:
        return 2
    elif gpt_rank <= cumulative_distribution.iloc[2]:
        return 3
    else:
        return 4

mapped_confidence_levels = np.array([map_quantile_to_level(rank) for rank in gpt_ranks])

for gpt_score, mapped_level in zip(gpt_confidence_scores, mapped_confidence_levels):
    print(f"GPT Score: {gpt_score}, Mapped Confidence Level: {mapped_level}")

In [ ]:
df_combined.loc[:, 'pred_conf_level_mapped'] = mapped_confidence_levels
color = sns.color_palette("YlGnBu", as_cmap=True)(0.5)  # 0.5 gives the middle color

# Plot a histogram without black rims and with the custom color
sns.histplot(df_combined['pred_conf_level_mapped'], bins=4, kde=False, color=color, edgecolor='white')
plt.title('Distribution of GPT Confidence Scores')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')

plt.show()

In [ ]:
df_combined['syndromic_dx_certainty_int'] = df_combined['syndromic_dx_certainty'].astype(int)
df_combined['syndromic_dx_certainty_str'] = df_combined['syndromic_dx_certainty_int'].astype(str)
df_combined['syndromic_dx_certainty_str'] = pd.Categorical(df_combined['syndromic_dx_certainty_str'], 
                                                         categories=['1', '2', '3', '4'], 
                                                         ordered=True)
df_combined['pred_dx_certainty_int'] = df_combined['pred_conf_level_mapped'].astype(int)
df_combined['pred_dx_certainty_str'] = df_combined['pred_dx_certainty_int'].astype(str)
df_combined['pred_dx_certainty_str'] = pd.Categorical(df_combined['pred_dx_certainty_str'], 
                                                         categories=['1', '2', '3', '4'], 
                                                         ordered=True)
confusion_matrix_data = pd.crosstab(df_combined['syndromic_dx_certainty_str'], df_combined['pred_dx_certainty_str'])

plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=(10, 8))

ax = sns.heatmap(confusion_matrix_data, annot=True, cmap="YlGnBu", fmt="d", annot_kws={"size": 34})
ax.invert_yaxis()
plt.xlabel('GPT Confidence', fontsize=35)  
plt.ylabel('Physician Confidence', fontsize=35) 
plt.xticks(fontsize=32) 
plt.yticks(fontsize=32)  
colorbar = ax.collections[0].colorbar  
colorbar.ax.tick_params(labelsize=24) 

plt.show()

### Stratification Analysis

In [ ]:
kappa_scores_by_certainty = calculate_weighted_kappa_by_certainty(df_combined)

for certainty_level, kappa_score in kappa_scores_by_certainty.items():
    print(f"Certainty Level {certainty_level}: Weighted Kappa = {kappa_score:.4f}")

In [ ]:
kappa_df_gpt = pd.DataFrame(list(kappa_scores_by_certainty.items()), columns=['Certainty Level', 'Weighted Kappa'])

plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='Certainty Level', y='Weighted Kappa', data=kappa_df_gpt, palette='YlGnBu', dodge=False)

plt.xlabel('GPT Confidence Score', fontsize=32)
plt.ylabel('Weighted Kappa Score', fontsize=32)
plt.xticks(fontsize=30)  
plt.yticks(fontsize=30)  
ax.set_ylim(0, 1.10)  

for i, row in kappa_df_gpt.iterrows():
    ax.text(i, row['Weighted Kappa'] + 0.005, f'{row["Weighted Kappa"]:.2f}', ha='center', va='bottom', fontsize=30)

plt.show()

# Revision - 5 Classes

## Load data and process for 5 classes experiment

In [ ]:
def extract_syndromic_dx(response):
    # Updated regex pattern to include new categories
    pattern = r'\*\*syndromic diagnosis:\*\*\s*(Normal vs MCI|MCI vs Dementia|Normal|MCI|Dementia)'
    match = re.search(pattern, response, re.IGNORECASE)
    if match:
        return match.group(1).lower()  # Normalize to lowercase
    else:
        return None
    
# Function to load all JSON files and combine them into a single DataFrame
def load_patient_data(json_directory):
    responses = []
    predicted_dx = []
    patient_ids = []
    # Iterate over all JSON files in the specified directory
    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            filepath = os.path.join(json_directory, filename)
            with open(filepath, 'r') as file:
                data_dict = json.load(file)
                for patient_id, result in data_dict.items():
                    responses.append(result)
                    predicted_dx.append(extract_syndromic_dx(result))
                    patient_ids.append(patient_id)
    # Combine all the loaded data into a single DataFrame
    combined_data = pd.DataFrame(
        {"PatientID": patient_ids,
        "ResponseTXT": responses,
        "predicted_dx": predicted_dx}
    )

    return combined_data


In [ ]:
df_result = load_patient_data(".../Results - sydronmic dx/GPT4o - Attempt 8 - Revision")

In [ ]:
df_dx = pd.read_csv('../../EDW Queries/Moura et al data-all-notes/patient_dx.csv')
df_combined = df_result.merge(df_dx, on='PatientID', how='inner')

In [ ]:
def pred_dx_to_category(value):
    if value == 'normal':
        return 'Normal'
    elif value == 'normal vs mci':
        return 'Normal-to-MCI'
    elif value == 'mci':
        return 'MCI'
    elif value == 'mci vs dementia':
        return 'MCI-to-Dementia'
    elif value == 'dementia':
        return 'Dementia'
    
def actual_dx_to_category(value):
    if value == 0:
        return 'Normal'
    elif value == 1:
        return 'Normal-to-MCI'
    elif value == 2:
        return 'MCI'
    elif value == 3:
        return 'MCI-to-Dementia'
    elif value == 4:
        return 'Dementia'

    
# Apply the function to the 'cdr' column and create a new 'cdr_category' column
df_combined['actual_category'] = df_combined['syndromic_dx'].apply(actual_dx_to_category)

df_combined['predicted_category'] = df_combined['predicted_dx'].apply(pred_dx_to_category)

In [ ]:
df_combined['actual_category'].value_counts()

## Analyses

### Performance Evaluation

In [ ]:
all_labels = ['Normal','Normal-to-MCI','MCI','MCI-to-Dementia','Dementia']

# Compute the confusion matrix
cm = confusion_matrix(df_combined['actual_category'], df_combined['predicted_category'], labels=all_labels)

# Normalize the confusion matrix by dividing each row by the sum of the row
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_normalized

In [ ]:
# Create annotation strings for counts and percentages
annot = np.array([["{0}\n({1:.1%})".format(count, perc) for count, perc in zip(row_counts, row_percents)] 
                  for row_counts, row_percents in zip(cm, cm_normalized)])

cm_df = pd.DataFrame(cm, index=all_labels, columns=all_labels)
# Set font style
plt.rcParams['font.family'] = 'Arial'

plt.figure(figsize=(16, 13))
ax = sns.heatmap(cm, annot=annot, cmap="YlGnBu", fmt="", annot_kws={"size": 32}, xticklabels=all_labels, yticklabels=all_labels)

plt.xlabel('Predicted', fontsize=35) 
plt.ylabel('Actual', fontsize=35) 
plt.xticks(fontsize=24)  
plt.yticks(fontsize=24)  
colorbar = ax.collections[0].colorbar 
colorbar.ax.tick_params(labelsize=24)  

plt.show()

In [ ]:
# Extract true and predicted labels
y_true = df_combined['actual_category']
y_pred = df_combined['predicted_category']

# Generate classification report
report = classification_report(
    y_true, 
    y_pred, 
    labels=['Normal','Normal-to-MCI','MCI','MCI-to-Dementia','Dementia'],  # ensure consistent order
    output_dict=True
)

report_df = pd.DataFrame(report).transpose()

# Display the per-class metrics
print(report_df[['precision', 'recall', 'f1-score', 'support']])

In [ ]:
label_mapping = {'Normal': 0, 'MCI': 2, 'Dementia': 4, 'Normal-to-MCI': 1, 'MCI-to-Dementia': 3}

# Use .loc to avoid SettingWithCopyWarning
df_combined.loc[:, 'actual_mapped'] = df_combined['actual_category'].map(label_mapping)
df_combined.loc[:, 'predicted_mapped'] = df_combined['predicted_category'].map(label_mapping)

# Calculate Quadratic Weighted Kappa
qwk = cohen_kappa_score(df_combined['actual_mapped'], df_combined['predicted_mapped'], weights='quadratic')

print(f"Quadratic Weighted Kappa: {qwk}")

### Adjudicator Confidence

In [ ]:
df_combined = df_combined.merge(df_certainty, on='PatientID', how='left')

In [ ]:
normal_df = df_combined[df_combined['actual_category'] == 'Normal']
print(normal_df.syndromic_dx_certainty.mean())
normalmci_df = df_combined[df_combined['actual_category'] == 'Normal-to-MCI']
print(normalmci_df.syndromic_dx_certainty.mean())
mci_df = df_combined[df_combined['actual_category'] == 'MCI']
print(mci_df.syndromic_dx_certainty.mean())
mcidementia_df = df_combined[df_combined['actual_category'] == 'MCI-to-Dementia']
print(mcidementia_df.syndromic_dx_certainty.mean())
dem_df = df_combined[df_combined['actual_category'] == 'Dementia']
print(dem_df.syndromic_dx_certainty.mean())

In [ ]:
certainty_ls = [
    3.215909090909091,
    1.565217391304348,
    2.596774193548387,
    2.090909090909091,
    3.6339285714285716
]
levels = ['CU', 'Normal vs. MCI', 'MCI', 'MCI vs. Dementia', 'Dementia']

In [ ]:
certainty_df = pd.DataFrame({
    'CI Stage': levels,
    'Mean Certainty Level': certainty_ls
})
plt.rcParams['font.family'] = 'Arial'

plt.figure(figsize=(12, 10))
ax = sns.barplot(x='CI Stage', y='Mean Certainty Level', data=certainty_df, hue='Mean Certainty Level', palette='YlGnBu', dodge=False)
ax.legend_.remove()
plt.xlabel('Syndromic Dx', fontsize=32)
plt.ylabel('Mean Confidence Level', fontsize=32)
plt.xticks(fontsize=30, rotation=30)
plt.yticks(fontsize=30)
ax.set_ylim(0, 4.0)  

# Add text labels on each bar
for i, row in certainty_df.iterrows():
    ax.text(i, row['Mean Certainty Level'] + 0.05, f'{row["Mean Certainty Level"]:.2f}', 
            ha='center', va='bottom', fontsize=30)

plt.tight_layout()
plt.show()